# Transformers

## 0. imports

In [1]:
%load_ext jupyter_black

In [4]:
import os
import re
import glob

import numpy as np
import pandas as pd

import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

## 1. Dataset

### 1.1 ETT Dataset

In [22]:
seq_len = 96
label_len = 48
pred_len = 96

flag = "train"
type_map = {"train": 0, "val": 1, "test": 2}
set_type = type_map[flag]

# 'forecasting task, options:[M, S, MS];
#  M:multivariate predict multivariate,
#  S:univariate predict univariate,
#  MS:multivariate predict univariate'
task = "M"
use_scale = True
timeenc = 1
freq = "h"
target = "OT"

data_path = "../data/ETT-small/ETTh1.csv"

In [8]:
border1s = [0, 12 * 30 * 24 - seq_len, 12 * 30 * 24 + 4 * 30 * 24 - seq_len]
border2s = [12 * 30 * 24, 12 * 30 * 24 + 4 * 30 * 24, 12 * 30 * 24 + 8 * 30 * 24]
border1 = border1s[set_type]
border2 = border2s[set_type]

In [18]:
df = pd.read_csv(data_path)
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")

if task == "S":
    df = df[[target]]

In [28]:
scaler = StandardScaler()
if use_scale:
    train_df = df[border1:border2]
    scaler.fit(train_df.values)

    data = scaler.transform(df.values)
else:
    data = df.values

In [40]:
stamp_df = pd.DataFrame()
stamp_df["date"] = df[border1:border2].index

In [41]:
stamp_df['mon']

,date
0,2016-07-01 00:00:00
1,2016-07-01 01:00:00
2,2016-07-01 02:00:00
3,2016-07-01 03:00:00
4,2016-07-01 04:00:00
...,...
8635,2017-06-25 19:00:00
8636,2017-06-25 20:00:00
8637,2017-06-25 21:00:00
8638,2017-06-25 22:00:00
